Importing required libraries for the code 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, cohen_kappa_score, f1_score, confusion_matrix
import itertools

Loading the dataset

In [ ]:
set1 = pd.read_csv('9070_training_af.csv')  # Training set
set2 = pd.read_csv('3024_internal validation_af.csv')  # Internal validation set
set3 = pd.read_csv('3024_external_validation_af.csv') # External validation set
data = pd.read_csv("dataset_merge_for_cv.csv")  # Merged Training and Internal validation set for cross validation 

In [ ]:
X_train = set1.drop(columns=['Smiles', 'Activity'])
Y_train = set1['Activity'].map({'Active': 1, 'Inactive': 0})

y_val = set2.drop(columns=['Smiles', 'Activity'])
y_val_labels = set2['Activity'].map({'Active': 1, 'Inactive': 0})

z_test = set3.drop(columns=['Smiles', 'Activity'])
z_test_labels = set3['Activity'].map({'Active': 1, 'Inactive': 0})

X = data.drop(columns=['Smiles', 'Activity'])
Y = data['Activity'].map({'Active': 1, 'Inactive': 0})

In [ ]:
print("Shape of X_train:", X_train.shape)
print("Shape of Y_train:", Y_train.shape)
print("Shape of y_val:", y_val.shape)
print("Shape of y_val_labels:", y_val_labels.shape)
print("Shape of z_test:", z_test.shape)
print("Shape of z_test_labels:", z_test_labels.shape)

print("Shape of X:", X.shape)
print("Shape of Y:", Y.shape)

Defining hyperparameter combinations

In [ ]:
hidden_layer_sizes = [(150, 100, 50), (50, 50), (100, 100), (100,)]
activations = ['relu', 'tanh']
alphas = [0.0001, 0.001, 0.01]
learning_rates = ['constant','adaptive']

# Creating all combinations of parameters
param_combinations = list(itertools.product(hidden_layer_sizes, activations, alphas, learning_rates))

Initializing variables to track the best model

In [ ]:
best_model = None
best_accuracy = 0
best_params = None

accuracy_data = []

Training and Evaluating each model

In [ ]:
for idx, params in enumerate(param_combinations):
    print(f"Training model {idx+1}/{len(param_combinations)} with params: {params}", flush=True)
    hls, activation, alpha, lr = params

    model = MLPClassifier(
        hidden_layer_sizes=hls,
        activation=activation,
        alpha=alpha,
        learning_rate=lr,
        random_state=42,
        verbose=True  
        )
        
    model.fit(X_train, Y_train)
        
    y_val_predictions = model.predict(y_val)
    accuracy = accuracy_score(y_val_labels, y_val_predictions)
        
    accuracy_data.append({
        'hidden_layer_sizes': hls,
        'activation': activation,
        'alpha': alpha,
        'learning_rate': lr,
        'accuracy': accuracy
    })

    # Tracking the best model
    if accuracy > best_accuracy:
        best_model = model
        best_accuracy = accuracy
        best_params = params
    
print("Best Parameters:", best_params)
print("Best Internal Validation Accuracy:", best_accuracy)

accuracy_df = pd.DataFrame(accuracy_data)
accuracy_df.to_csv('model_accuracies.csv', index=False)
print("\nModel accuracy data has been saved to 'model_accuracies.csv'.")

Evaluating the performance of the best model on external validation dataset

In [ ]:
z_test_predictions = best_model.predict(z_test)
z_test_probabilities = best_model.predict_proba(z_test)[:, 1]  # Probabilities for ROC AUC score

accuracy = accuracy_score(z_test_labels, z_test_predictions)
precision = precision_score(z_test_labels, z_test_predictions)
recall = recall_score(z_test_labels, z_test_predictions)
f1 = f1_score(z_test_labels, z_test_predictions)
roc_auc = roc_auc_score(z_test_labels, z_test_probabilities)

evaluation_metrics = {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'roc_auc': roc_auc
}

print("\nFinal Model Evaluation on External Validation Set (z_test):")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")

evaluation_df = pd.DataFrame([evaluation_metrics])
evaluation_df.to_csv('best_model_evaluation.csv', index=False)
print("\nBest model's evaluation metrics have been saved to 'best_model_evaluation.csv'.")

Plotting the ROC AUC Curve for the final MLP model

In [ ]:
fpr, tpr, thresholds = roc_curve(z_test_labels, z_test_probabilities)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.title('ROC AUC Curve for Best Model on External Validation Set')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.grid()
plt.savefig('roc_auc_curve.png')
plt.show()


10-Fold Cross Validation for the final MLP model

Variable Initialization

In [2]:
hls, activation, alpha, lr = best_params

all_seed_results = []

Code for running the cross validation

In [ ]:
for seed in range(10):
    print(f"\nStarting 10-fold CV with random seed {seed}")
    kf = KFold(n_splits=10, shuffle=True, random_state=seed)
    
    seed_metrics = {
        'accuracy': [],
        'precision': [],
        'sensitivity': [],
        'specificity': [],
        'cohen_kappa': [],
        'f1_score': [],
        'auroc': []
    }
    
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
        
        model = MLPClassifier(
            hidden_layer_sizes=hls,
            activation=activation,
            alpha=alpha,
            learning_rate=lr,
            random_state=seed
        )
        
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1]
        
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, zero_division=0)
        sensitivity = recall_score(y_test, y_pred) 
        f1 = f1_score(y_test, y_pred)
        auroc = roc_auc_score(y_test, y_prob)
        cohen_kappa = cohen_kappa_score(y_test, y_pred)
        
        # Calculating specificity
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        
        seed_metrics['accuracy'].append(accuracy)
        seed_metrics['precision'].append(precision)
        seed_metrics['sensitivity'].append(sensitivity)
        seed_metrics['specificity'].append(specificity)
        seed_metrics['cohen_kappa'].append(cohen_kappa)
        seed_metrics['f1_score'].append(f1)
        seed_metrics['auroc'].append(auroc)
        
        print(f"Seed {seed}, Fold {fold + 1}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}, Cohen’s Kappa: {cohen_kappa:.4f}, F1-Score: {f1:.4f}, AUROC: {auroc:.4f}")
    
    # Calculate mean and standard deviation for each metric
    seed_results = {
        'seed': seed,
        'accuracy_mean': np.mean(seed_metrics['accuracy']),
        'accuracy_std': np.std(seed_metrics['accuracy']),
        'precision_mean': np.mean(seed_metrics['precision']),
        'precision_std': np.std(seed_metrics['precision']),
        'sensitivity_mean': np.mean(seed_metrics['sensitivity']),
        'sensitivity_std': np.std(seed_metrics['sensitivity']),
        'specificity_mean': np.mean(seed_metrics['specificity']),
        'specificity_std': np.std(seed_metrics['specificity']),
        'cohen_kappa_mean': np.mean(seed_metrics['cohen_kappa']),
        'cohen_kappa_std': np.std(seed_metrics['cohen_kappa']),
        'f1_score_mean': np.mean(seed_metrics['f1_score']),
        'f1_score_std': np.std(seed_metrics['f1_score']),
        'auroc_mean': np.mean(seed_metrics['auroc']),
        'auroc_std': np.std(seed_metrics['auroc'])
    }
    
    print(f"\nSeed {seed} Results:")
    for metric, value in seed_results.items():
        if metric != 'seed':
            print(f"{metric}: {value:.4f}")
    
    all_seed_results.append(seed_results)

results_df = pd.DataFrame(all_seed_results)
results_df.to_csv("cv_results_with_metrics.csv", index=False)
print("\nCross-validation results with metrics have been saved to 'cv_results_with_metrics.csv'.")
